# L10: Lettuce Operational Semantics
* This document is subject to change before Lecture

## Overview
* The Lettuce AST
* Well Formed vs ILL Formed expressions
* Interpreter

## The Lettuce AST

### Grammar for lettuce
We are now ready to define a grammar for Lettuce. Here is a part of the grammar

$$\begin{array}{rcll}
\mathbf{Expr} & \rightarrow &  Value \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
\mathbf{Value} & \rightarrow & Const(\mathbf{Number}) \\
 & | & True \\
 & | & False \\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) & \text{function (identifier) expr } \\ 
\mathbf{Identifier} & \rightarrow & String \\
\mathbf{Number} & \rightarrow & Double \\
\end{array}$$

It helps to how programs written in the concrete syntax translate into abstract syntax using the grammar.

#### Example 1

~~~
let x = 10 + 15 in 
    x + 35
~~~

will translate to 

~~~
Let("x", Plus(Const(10), Const(15)), 
         Plus(Ident("x"), Const("35")
    )
~~~

#### Example 2

~~~
let dog = 5 in 
    let cat = dog + dog in
        cat + dog
~~~

will translate to 

~~~
Let("dog", Const(5),
           Let("cat", Plus(Ident("dog"), Ident("dog"),
                      Plus(Ident("cat"), Ident("dog"))
              )
    )
~~~

           
### Writing the AST
Shall we write the AST Together?

In [2]:
// Expr
// Ident(Identifier)
// Plus(Expr, Expr)
// Let(Identifier, Expr, Expr)

// Value
// Const(Number)
// True
// False

// Identifier
type Identifier = String

// Number
type Number = Double

defined type Identifier
defined type Number

#### Examples for the AST
Can we now write Scala code representing our ASTs from the above examples?

#### Example 1

~~~
let x = 10 + 15 in 
    x + 35
~~~

will translate to 

~~~
Let("x", Plus(Const(10), Const(15)), 
         Plus(Ident("x"), Const("35")
    )
~~~

In [ ]:
val ex1:Expr = ???

#### Example 2

~~~
let dog = 5 in 
    let cat = dog + dog in
        cat + dog
~~~

will translate to 

~~~
Let("dog", Const(5),
           Let("cat", Plus(Ident("dog"), Ident("dog"),
                      Plus(Ident("cat"), Ident("dog"))
              )
    )
~~~

In [ ]:
val ex2:Expr = ???

## Well Formed vs ILL Formed expressions
__Fact:__ Abstract Syntax Trees are built from Context Free Grammars. Unfortunately, due to limitations arising
from context freedom (i.e, LHS of each rule is just a single nonterminal), we cannot really enforce rules like 
- Variables should be declared before use.
- Boolean expressions cannot be added to integer expressions
- Function calls can only be made over proper functions.

Programming languages generally have different enforcement mechanisms:
- Check if a program is _well-formed_ while parsing the program.
- Check types of expressions while parsing the program (static type checking).
- Check types of expressions while interpreting/evaluating the program (runtime typechecking).
- A combination of above.

Consider our definition of the language lettuce.
* What are some strange programs that can be built based on the grammar itself?
* What are some valid ASTs that don’t make any sense in code?

### Example 1
* AST: TopLevel(Plus(True, Const(5)))
* Concrete Lettuce Syntax: true + 5
* What is the value of such an expression?
* Is there a way to get a value?
    * We could maybe do some type casting and get this to work
    * We could just throw errors on this during evaluation (dynamic time)
    * We could throw and error on this before ever evaluating it, perhaps by checking type safety (static time)
* What does Scala do with this expression?

In [2]:
true + 5.0

cmd2.sc:1: type mismatch;
 found   : Double(5.0)
 required: String
val res2 = true + 5.0
                  ^Compilation Failed

: 

* What does JavaScript do with this expression?

### Example 2
* AST: Ident(“dog”)
* Concrete: dog
* What would Scala do?

In [2]:
dog

cmd2.sc:1: not found: value dog
val res2 = dog
           ^Compilation Failed

: 

* What does JavaScript do with this expression?

### Additional Examples

* AST: ???
* Concrete: ???
* What would Scala do?

* AST: ???
* Concrete: ???
* What would Scala do?

* AST: ???
* Concrete: ???
* What would Scala do?

We will have our interpreter catch errors in Lettuce programs at compile time. 
- Variables should be declared before use (we can catch this using a special declare before use check).
- Boolean expressions cannot be added to integer expressions (we will catch this when we do type checking).
- Function calls can only be made over proper functions (we will catch this during type checking).

### Review of Inference Rulse
* Now for our inference rules. The inference rules leverage the operational semantic and some inductive logic to create strong documentation of the code.
    * They are also helpful if you ever need to **PROVE** anything about your code
        * not to worry, we aren’t proving anything in this course…
* General pattern for Inference Rule: 

$$\begin{array}{c}
\text{0-or-many premises that must hold true for the conclusion to hold true} \\
\hline
\text{conclusions that can be drawn} \\
\end{array}\ \text{(Option to Name the Rule)}$$

* Or if you prefer

$$\begin{array}{c}
\text{P0}
\hspace{1cm}
\text{P1}
\hspace{1cm}
\text{...}
\hspace{1cm}
\text{Pn-1}
\hspace{1cm}
\text{Pn} \\
\hline
\text{C} \\
\end{array}\ \text{Name}$$

### Lettuce inference rules for Well Formed programs
Let us define inference rules that will help us check if an expression is checked before use. 

* Operational Semantic: $wellFormed(\texttt{e}, S)$
    * “e” is an expression
    * “S” is the set of variables currently defined (the set of bound variables)
    * This implies a Boolean return value
    * If an expression is not explicitly well formed according to the inference rules then the expression is implicitly ill formed
    * let’s code it (see below)

* Inference Rules:
    * $$ \begin{array}{c}
e \in \{True, False, Const(n)\} \\
\hline
wellFormed(\texttt{e}, S) \\
\end{array} \text{wellFormedValue} $$
        * English : any expression e is well formed in set S, so long as the expression e is True, False, or a Const(n)
        * Meaning: numbers and Booleans are well formed
        * let’s code it (see below)
    * $$\begin{array}{c}
wellFormed(\texttt{e1}, S) \;\;\; wellFormed(\texttt{e2}, S \cup \{ x\} ) \\
\hline
wellFormed(\texttt{Let(x, e1, e2)}, S) \\
\end{array} \text{wellFormedLet} $$
        * English: Any let over “x”, “e1”, “e2” in set “S” is only well formed if “e1” is wellfomed in set “S” and if “e2” is well formed in set “S” joined with “x”.
        * Meaning: For a let binding to be valid, the binding expression “e1” must be valid. Also the body expression “e2” must be valid in the parent set “S” extended to include our variable “x”.
        * let’s code it (see below)
    * $$\begin{array}{c}
x \in S \\
\hline
wellFormed(\texttt{Ident(x)}, S) \\
\end{array} \text{wellFormedIdent} $$
        * English: Any identifier “x” in set “S” is well formed iff “x” exists in “S”.
        * Meaning: For an identifier to be well formed it must already exist in the set of bound variable
        * let’s code it (see below)

In [ ]:
// wellFormed(e, S)
// wellFormedValue
// wellFormedLet
// wellFormedIdent

* Given the partial implementation of wellFormed that we have so far, what sort of expressions ought to be well formed?

In [ ]:
val e0: Expr = ???
assert(wellFormed(e0))
val e1: Expr = ???
assert(wellFormed(e1))
val e2: Expr = ???
assert(wellFormed(e2))

* Given the partial implementation of wellFormed that we have so far, what sort of expressions ought to be ill formed (not well formed)?

In [9]:
def illFormed(e:Expr):Boolean = !wellFormed(e, Set())

val e3: Expr = ???
assert(wellFormed(e3))
val e4: Expr = ???
assert(wellFormed(e4))
val e5: Expr = ???
assert(wellFormed(e5))

cmd9.sc:4: not enough arguments for method wellFormed: (e: ammonite.$sess.cmd7.wrapper.cmd4.Expr, S: Set[String])Boolean.
Unspecified value parameter S.
val res9_2 = assert(wellFormed(e3))
                              ^cmd9.sc:6: not enough arguments for method wellFormed: (e: ammonite.$sess.cmd7.wrapper.cmd4.Expr, S: Set[String])Boolean.
Unspecified value parameter S.
val res9_4 = assert(wellFormed(e4))
                              ^cmd9.sc:8: not enough arguments for method wellFormed: (e: ammonite.$sess.cmd7.wrapper.cmd4.Expr, S: Set[String])Boolean.
Unspecified value parameter S.
val res9_6 = assert(wellFormed(e5))
                              ^Compilation Failed

: 

* What is the wellFormed function? 
    * It is a static checker
    * It checks at static time if the expression is valid
        * static time == compilation time == before evaluation
* See the course notes for more details

## Evaluation of Lettuce
* Let’s write an interpreter for the Lettuce language.

### Operational Semantic (Op. Sem.)
* First we’ll define an operational sematic
    * If I were a programming linguist I would write the op. sem.: $\sigma \models \texttt{e} \Downarrow v$
        * this means: that an expression “e” is evaluated to a value “v” in 0-or-many steps inside a variable environment sigma ($\sigma$)
    * But I’m not a programming linguist and I don’t love this notation instead, Ill write: “v = eval(e, env)”
    * What is env? Why is it useful? We didn’t need this for “Logic” or “Maths”, what could this be?
    * “env” – often $\sigma$ - is our variable environment. 
        * It maps Identifiers to values
            * …for now. Later on we’ll allow it to map to locations, expressions and values.
        * environments map variables “x” to values “v”
* Let’s look at the function definition
* Op. Sem.: v = eval(e, env)

In [ ]:
def eval0(e:Expr, env:Map[String, Value] = Map()): Value = ???

* Observations
    * e: Expr, nothing new there
    * The type of env is “Map[String, Value]”
        * Map[A,B] is a Scala type – often denoted Map[K,V] because key-value pairs...
        * It’s an awful lot like python dictionary, or JavaScript Objects
        * Generally I would call these a variant of an “associative array”
    * The env has a default value "Map()" - the empty map
    * the return type is a “Value” – a subset of all “Expr”
        * Unlike Maths and Logic, Lettuce has many types in it’s value domain
        * We can no longer return simple Scala types like “Int” and “Boolean”

### Inference Rules
* Now for some inference rules!
* I’ll write a rule for True, and ask you to write your own for False and Const(n)

* True
    * $$\begin{array} & \\
\hline
True\ =\ eval(True,\ env)\\
\end{array} \text{evalTrue}$$

* False
    * $$\begin{array} & 
??? \\
\hline
??? \\
\end{array} \text{evalFalse}$$

* Const
    * $$\begin{array} & 
??? \\
\hline
??? \\
\end{array} \text{evalConst}$$

* Let’s code it

In [ ]:
def eval1(e:Expr, env:Map[String, Value]): Value = e match {
    // evalTrue
    // evalFalse
    // evalConst
    case _ => ???
}

* Can we combine our 3 eval inference rules?
    * $$\begin{array} & 
??? \\
\hline
??? \\
\end{array} \text{evalValue}$$

In [ ]:
def eval2(e:Expr, env:Map[String, Value]): Value = e match {
    // evalVal
    case _ => ???
}

* Here is an inference rule for Ident
    * $$\begin{array} &
v = env(x) \\
\hline
v = eval(Ident(x), env)\\
\end{array} {evalIdent}$$

In [ ]:
def eval3(e:Expr, env:Map[String, Value]): Value = e match {
    // evalVal
    case _ => ???
}

* Now for some more interesting stuff. Let us consider the let bindings – we’ll start with a stupid statement
    * Expr: Let(“x”, True, False)
        * Value: False
        * Concrete Lettuce: let x = true in false
        * Lettuce value: false
        * Steps: 
            * eval(let x = true in false, {}) -->
            * eval(false, { “x” -> true } ) --> 
            * true
    * Expr: Let(“x”, True, Ident(“x”))
        * Value: True
        * Concrete Lettuce: let x = true in x
        * Lettuce value: true
        * Steps: 
            * eval(let x = true in x, {}) -->
            * eval(x, { “x” -> true } ) --> 
            * eval(true, { “x” -> true })--> 
            * true

* Are we ready to write an inference rule of let bindings using our operational sematic?
    * Or at least some part of it?
    * Let’s use the chalkboards…
    * op. sem.: v = eval(e, env)
    * $$ \begin{array} &
??? \\
\hline
??? \\
\end{array} \text{evalLet} $$

## Overview
* The Lettuce AST
* Well Formed vs ILL Formed expressions
* Interpreter

## TODOs
* Homework and Quiz 4 are due Friday
* Project 1 is due March 1